In [1]:
!nvidia-smi

Mon Feb 19 13:32:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:03:00.0 Off |                  N/A |
|  0%   45C    P8              20W / 250W |      1MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR
import os
from PIL import Image
import numpy as np
import random
from tqdm import tqdm

from base64 import b64encode
import numpy as np
import json
import cv2
import subprocess

# For video display:
from IPython.display import HTML
from pathlib import Path
from torch import autocast
from torchvision import transforms as tfms
import pytorch_lightning as pl
from transformers import logging #CLIPTextModel, CLIPTokenizer,
from transformers import BlipProcessor, BlipForConditionalGeneration

# torch.manual_seed(1)

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

from huggingface_hub import hf_hub_download
from diffusers import DiffusionPipeline

/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [18]:
os.getcwd()

'/'

In [19]:
# !git clone https://github.com/lllyasviel/ControlNet.git
!git clone https://github.com/anilbhatt1/ControlNet.git

Cloning into 'ControlNet'...
remote: Enumerating objects: 1398, done.
remote: Counting objects: 100% (483/483), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 1398 (delta 425), reused 408 (delta 393), pack-reused 915
Receiving objects: 100% (1398/1398), 122.40 MiB | 27.55 MiB/s, done.
Resolving deltas: 100% (637/637), done.
Updating files: 100% (510/510), done.


In [5]:
# Do huggingface-cli login and add token via vastai terminal before doing this
downloaded_model_path = hf_hub_download(repo_id="runwayml/stable-diffusion-v1-5",
                                        filename="v1-5-pruned.ckpt",
                                        use_auth_token=True)
print(downloaded_model_path)

/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/1d0c4ebf6ff58a5caecab40fa1406526bca4b5b9/v1-5-pruned.ckpt


In [6]:
os.getcwd()

'/'

In [ ]:
#Copy file using gdown and apt install unzip before running this
!unzip -q '/fill50k.zip'

In [8]:
!ls '/fill50k/source' | wc -l
!ls '/fill50k/target' | wc -l

50000
50000


In [9]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = []
        with open('/prompt_10k.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))
        self.width = 512
        self.height = 512

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        source = cv2.imread('/fill50k/' + source_filename)
        target = cv2.imread('/fill50k/' + target_filename)

        source = cv2.resize(source, (self.width, self.height), interpolation=cv2.INTER_NEAREST)
        target = cv2.resize(target, (self.width, self.height), interpolation=cv2.INTER_NEAREST)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)

In [10]:
dataset = MyDataset()
print(len(dataset))

item = dataset[5234]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

10000
fuchsia circle with yellow background
(512, 512, 3)
(512, 512, 3)


In [12]:
!mkdir '/models'

In [13]:
!cp '/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/1d0c4ebf6ff58a5caecab40fa1406526bca4b5b9/v1-5-pruned.ckpt' '/models'

In [20]:
%cd '/ControlNet'
! ls

/ControlNet
LICENSE			       gradio_pose2image.py
README.md		       gradio_scribble2image.py
annotator		       gradio_scribble2image_interactive.py
cldm			       gradio_seg2image.py
config.py		       ldm
docs			       models
environment.yaml	       share.py
font			       test_imgs
github_page		       tool_add_control.py
gradio_annotator.py	       tool_add_control_sd21.py
gradio_canny2image.py	       tool_transfer_control.py
gradio_depth2image.py	       tutorial_dataset.py
gradio_fake_scribble2image.py  tutorial_dataset_test.py
gradio_hed2image.py	       tutorial_train.py
gradio_hough2image.py	       tutorial_train_sd21.py
gradio_normal2image.py


In [21]:
!python3 tool_add_control.py '/models/v1-5-pruned.ckpt' '/models/control_sd15_ini.ckpt'

logging improved.
create_model-config : {'model': {'target': 'cldm.cldm.ControlLDM', 'params': {'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'control_key': 'hint', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'only_mid_control': False, 'control_stage_config': {'target': 'cldm.cldm.ControlNet', 'params': {'image_size': 32, 'in_channels': 4, 'hint_channels': 3, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_checkpoint': True, 'legacy': False}}, 'unet_config': {'target': 'cldm.cldm.ControlledUnetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320,

In [22]:
os.chdir('/ControlNet')
os.getcwd()

'/ControlNet'

In [23]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from tutorial_dataset import MyDataset
from cldm.logger import ImageLogger

In [24]:
from cldm.model import create_model, load_state_dict

In [25]:
resume_path = '/models/control_sd15_ini.ckpt'
batch_size = 1
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False
# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v15.yaml').cpu()

create_model-config : {'model': {'target': 'cldm.cldm.ControlLDM', 'params': {'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'control_key': 'hint', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'only_mid_control': False, 'control_stage_config': {'target': 'cldm.cldm.ControlNet', 'params': {'image_size': 32, 'in_channels': 4, 'hint_channels': 3, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_checkpoint': True, 'legacy': False}}, 'unet_config': {'target': 'cldm.cldm.ControlledUnetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolu

In [26]:
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

Loaded state_dict from [/models/control_sd15_ini.ckpt]


In [29]:
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(accelerator="auto", precision=32, callbacks=[logger], max_epochs=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, dataloader)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3080 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /ControlNet/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | model             | DiffusionWrapper   | 859 M 
1 | first_stage_model | AutoencoderKL      | 83.7 M
2 | cond_stage_model  | FrozenCLIPEmbedder | 123 M 
3 | control_model     | ControlNet         | 361 M 
--------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
